# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096893


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:02<00:38,  1.37s/it]

Rendering models:  13%|█▎        | 4/30 [00:06<00:41,  1.59s/it]

Rendering models:  20%|██        | 6/30 [00:07<00:27,  1.13s/it]

Rendering models:  27%|██▋       | 8/30 [00:07<00:17,  1.23it/s]

...

Rendering models:  77%|███████▋  | 23/30 [00:08<00:00,  7.21it/s]

Rendering models:  83%|████████▎ | 25/30 [00:08<00:00,  8.44it/s]

Rendering models:  90%|█████████ | 27/30 [00:08<00:00,  9.92it/s]

Rendering models: 100%|██████████| 30/30 [00:08<00:00, 11.52it/s]

Jnursssmith5263                       0.000200
jnarayanbvg                           0.000168
not-logged-in-c6917d9b19f41e6c2995    0.006899
awright5                              0.000873
not-logged-in-74261f68b83c7ae82ce3    0.000176
AaronWallage117                       0.000314
salmi.a01                             0.000413
not-logged-in-e5015a872e4a0f43792d    7.476447
rmz4160                               0.000195
bentleyghioda                         0.000139
lore98                                0.000120
Planetme                              0.004172
Saharisunshine                        0.000249
Tarskii                               0.000818
RaulE                                 0.000238
sallymcshane                          0.128032
Lavadude                              0.018418
not-logged-in-de31fdc547666aaf592d    0.000181
AspenTreez                            0.000561
Jassels2                              0.000792
lord_oles                             0.000139
FortnitEboi  

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())